In [2]:
import numpy as np
from qiskit import *

from qiskit.circuit import Gate
user_gate = Gate(name='user_gate', num_qubits=2, params=[])

In [3]:
qr = QuantumRegister(3, 'input')
qcirc = QuantumCircuit(qr)
qcirc.append(user_gate, [qr[0], qr[1]])
qcirc.append(user_gate, [qr[1], qr[2]])

qcirc.draw()

┌────────────┐              
input_0: ┤0           ├──────────────
         │  user_gate │┌────────────┐
input_1: ┤1           ├┤0           ├
         └────────────┘│  user_gate │
input_2: ──────────────┤1           ├
                       └────────────┘

In [7]:
## Composite Gates
# Building a SubCircuit 'sub_register'
sub_register = QuantumRegister(2)
sub_qcirc = QuantumCircuit(sub_register, name='sub_register')
sub_qcirc.h(sub_register[0])
sub_qcirc.crz(1, sub_register[0], sub_register[1])
sub_qcirc.barrier()
sub_qcirc.id(sub_register[1])
sub_qcirc.u3(1, 2, -2, sub_register[0])

## Convert to a gate and stick it onto a place in the bigger circuit.
sub_instruction = sub_qcirc.to_instruction() # This is later appended onto another circuit.

# Building a Circuit that we'll append the sub-circuit above to.

qr = QuantumRegister(3, 'input_state')
qcirc = QuantumCircuit(qr)
qcirc.h(qr[0]) # Applies a Hadamard Gate
qcirc.cx(qr[0], qr[1]) # Applies a CX Gate to Qubits 0 and 1.
qcirc.cx(qr[1], qr[2]) # Applies a CX Gate to Qubits 1 and 2.
qcirc.append(sub_instruction, [qr[1], qr[2]])

qcirc.draw()

┌───┐                           
input_state_0: ┤ H ├──■────────────────────────
               └───┘┌─┴─┐     ┌───────────────┐
input_state_1: ─────┤ X ├──■──┤0              ├
                    └───┘┌─┴─┐│  sub_register │
input_state_2: ──────────┤ X ├┤1              ├
                         └───┘└───────────────┘

In [8]:
# Let's Decompose the gates more.
decomposed_qcirc = qcirc.decompose() 
decomposed_qcirc.draw()

┌─────────┐                                         
input_state_0: ┤ U2(0,π) ├──■──────────────────────────────────────
               └─────────┘┌─┴─┐     ┌───┐          ░ ┌────────────┐
input_state_1: ───────────┤ X ├──■──┤ H ├────■─────░─┤ U3(1,2,-2) ├
                          └───┘┌─┴─┐└───┘┌───┴───┐ ░ └───┬───┬────┘
input_state_2: ────────────────┤ X ├─────┤ RZ(1) ├─░─────┤ I ├─────
                               └───┘     └───────┘ ░     └───┘

In [13]:
## Building Parameterized Circuits
from qiskit.circuit import Parameter
theta = Parameter('θ')
n = 5  #Iteration Variable

qc = QuantumCircuit(5, 1)
qc.h(0)

# Loops through and creates cnot gates on all the qubits (top to bottom).
for i in range(n-1):
    qc.cx(i, i+1)
    
qc.barrier()
qc.rz(theta, range(5)) # Places RZ Gates on all Qubits
qc.barrier()

# Loops through and creates cnot gates on all the qubits (bottom to top).
for i in reversed(range(n-1)):
    qc.cx(i, i+1)
qc.h(0)
qc.measure(0, 0)

qc.draw()
                                   #debug why qc.draw('mpl') isn't functioning.#

┌───┐                     ░ ┌───────┐ ░                     ┌───┐┌─┐
q_0: ┤ H ├──■──────────────────░─┤ RZ(θ) ├─░──────────────────■──┤ H ├┤M├
     └───┘┌─┴─┐                ░ ├───────┤ ░                ┌─┴─┐└───┘└╥┘
q_1: ─────┤ X ├──■─────────────░─┤ RZ(θ) ├─░─────────────■──┤ X ├──────╫─
          └───┘┌─┴─┐           ░ ├───────┤ ░           ┌─┴─┐└───┘      ║ 
q_2: ──────────┤ X ├──■────────░─┤ RZ(θ) ├─░────────■──┤ X ├───────────╫─
               └───┘┌─┴─┐      ░ ├───────┤ ░      ┌─┴─┐└───┘           ║ 
q_3: ───────────────┤ X ├──■───░─┤ RZ(θ) ├─░───■──┤ X ├────────────────╫─
                    └───┘┌─┴─┐ ░ ├───────┤ ░ ┌─┴─┐└───┘                ║ 
q_4: ────────────────────┤ X ├─░─┤ RZ(θ) ├─░─┤ X ├─────────────────────╫─
                         └───┘ ░ └───────┘ ░ └───┘                     ║ 
c: 1/══════════════════════════════════════════════════════════════════╩═
                                                                       0